In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Data Import and Preprocessing

In [5]:
train = pd.read_csv("/kaggle/input/Kannada-MNIST/train.csv")
train.head()

In [6]:
test = pd.read_csv("/kaggle/input/Kannada-MNIST/Dig-MNIST.csv")
test.head()

In [7]:
y_train = train.iloc[:,0].to_numpy() # converts label column to numpy array
x_train = train.iloc[:,1:].to_numpy() # converts data feature columns to numpy array

y_test = test.iloc[:,0].to_numpy()
x_test = test.iloc[:,1:].to_numpy()


In [8]:
x_train.shape

In [9]:
import matplotlib.pyplot as plt

one_image = x_train[50]

plt.imshow(one_image.reshape(28,28), cmap= "gray") #re-map 1d array to 2d array to draw image
y_train[50]

In [10]:
x_train = x_train/255
x_train - x_train.astype("float32")
x_test = x_test/255
x_test - x_test.astype("float32")

**Baseline Model**

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# create baseline model
baseline = Sequential([
    Dense(10, input_shape = (784, ), activation = "softmax")
])

# compile model
baseline.compile(loss = "sparse_categorical_crossentropy", # loss function = sparse categorical crossentropy
                optimizer = "adam",
                metrics = ["accuracy"])

# train model
history = baseline.fit(x_train, y_train, epochs = 10, validation_split = 0.1) # 10% data for validation




In [12]:
# check overfitting by plotting loss curvers
train_accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]

train_loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(len(train_accuracy))

plt.plot(epochs, train_accuracy, '-b')
plt.plot(epochs, val_accuracy, '-g')
plt.title("Accuracy Plot")
plt.figure()

plt.plot(epochs, train_loss, '-b')
plt.plot(epochs, val_loss, '-g')
plt.title("Loss Plot")
plt.figure()

Convolutional Neural Network (CNN)

In [26]:
# reshape data into 2 dimensional
x_train_2d = x_train.reshape(60000, 28, 28, 1) # reshape(# samples, height of sample, width of sample, channel dimension)
x_test_2d = x_test.reshape(-1, 28, 28, 1) # -1 means infer dimension

In [27]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3) # stops training when validation loss stops improving (over 3 epochs)

cnn = Sequential([
    Conv2D(filters = 32, kernel_size = (3,3), input_shape = (28,28,1), padding="same", activation = "relu"), # hyperparameters - can be tuned!
    MaxPooling2D(), 
    Conv2D(filters = 64, kernel_size = (3,3), padding="same", activation = "relu"), # add another convolutional layer - another hyperparameter
    MaxPooling2D(), 
    Flatten(), # flatten object from 2d to 1d
    Dense(256, activation = "relu"),
    Dense(10, activation = "softmax")
])

cnn.compile(loss="sparse_categorical_crossentropy",
           optimizer = "adam",
           metrics = ["accuracy"])

history = cnn.fit(x_train_2d,y_train,epochs = 20, validation_split = 0.1)# ,callbacks = [es])

In [28]:
cnn.evaluate(x_test_2d,y_test)

In [29]:
#submit to kaggle compeition
sample_submit = pd.read_csv("/kaggle/input/Kannada-MNIST/sample_submission.csv")
real_test = pd.read_csv("/kaggle/input/Kannada-MNIST/test.csv")

sample_submit.head()

In [31]:
real_test.head()

real_test = real_test.iloc[:, 1:].to_numpy()
real_test = real_test/255
real_test = real_test.astype("float32")
real_test = real_test.reshape(-1, 28,28,1)

In [32]:
predictions = cnn.predict(real_test)
predictions[:3]